#**Trabalho de casa 1** - SAT Solving
  
#### Número: **pg47483**
#### Nome: **Maria José Costa Ramos**
#### Curso:  **Mestrado em Engenharia Informática**
#### Disciplina: **Metodos Formais em Engenharia de Software**


In [ ]:
#Instalação da biblioteca PySAT

!pip install python-sat[pblib,aiger]

     |████████████████████████████████| 1.8 MB 5.0 MB/s 
     |████████████████████████████████| 3.4 MB 57.4 MB/s 
     |████████████████████████████████| 49 kB 6.4 MB/s 
     |████████████████████████████████| 106 kB 70.3 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42724 sha256=9659005c57e062fd4f35d6cf873bad1135234c3f677b94fc858fed2cdecfc0d9
  Stored in directory: /root/.cache/pip/wheels/88/5d/ba/f27d8af07306b65ee44f9d3f9cadea1db749a421a6db8a99bf
  Created wheel for pyrsistent: filename=pyrsistent-0.17.3-cp37-cp37m-linux_x86_64.whl size=98097 sha256=b62d6f14d0ba693f7b65b09fbe031b7098b65be5c570e3b2bf8c6c88e1398160
  Stored in directory: /root/.cache/pip/wheels/a5/52/bf/71258a1d7b3c8cbe1ee53f9314c6f65f20385481eaee573cc5
Successfully built parsimonious pyrsistent
  Attempting uninstall: pyrsistent
    Found existing installation: pyrsistent 0.18.0
    Uninstalling pyrsistent-0.18.0:
    

# Problema

O “Clube Desportivo de Ribeirão” tem as seguintes regras:  
* Todos os sócios que usam bigode são casados.  
* Cada sócio do clube que não é de Ribeirão tem que usar camisola amarela.  
* Os sócios casados não podem assistir aos jogos ao Domingo.
* Um sócio vai aos jogos ao Domingo se e só se é de Ribeirão.
* Cada sócio usa bigode ou não usa camisola amarela.
* Todos os sócios de Ribeirão usam bigode.


### 1. Por forma a codificar este puzzle como problema SAT, defina um conjunto adequado de variáveis proposicionais, exprima as regras acima como fórmulas proposicionais, e converta essas fórmulas para CNF.

### Variáveis
**B** - Tem bigode.  
**C** - É casado.  
**R** - É de Ribeirão.  
**A** - Usa camisola amarela.  
**D** - Assiste jogos aos domingos. 

### Cláusulas
* Todos os sócios que usam bigode são casados. **B → C**
* Cada sócio do clube que não é de Ribeirão tem que usar camisola amarela. **¬R → A**
* Os sócios casados não podem assistir aos jogos ao Domingo. **C → ¬D**
* Um sócio vai aos jogos ao Domingo se e só se é de Ribeirão. **D ⇔ R**
* Cada sócio usa bigode ou não usa camisola amarela. **B ∨ ¬A**
* Todos os sócios de Ribeirão usam bigode. **R → B**


### Cláusulas em CNF
* **¬B ∨ C**
* **R ∨ A**
* **¬C ∨ ¬D** 
* (D ∧ R) ∨ (¬D ∧ ¬R) ⇔ (D ∨ (¬D ∧ ¬R)) ∧ (R ∨ (¬D ∧ ¬R)) ⇔ (D ∨ ¬D) ∧ (D ∨ ¬R) ∧ (R ∨ ¬D) ∧ (R ∨ ¬R),   
que, simplificando, é equivalente a  **(D ∨ ¬R) ∧ (R ∨ ¬D)**
* **B ∨ ¬A**
* **¬R ∨ B**

### 2. Codifique o problema num SAT solver e comprove que o conjunto de regras é consistente.


In [ ]:
from pysat.solvers import Minisat22

bigode = 'tem bigode'
casado = 'é casado'
ribeirao = 'é de Ribeirão'
amarela = 'usa camisola amarela'
domingo = 'assiste jogos aos domingos'

clube = [bigode,casado,ribeirao,amarela,domingo]

variaveis = {}
i = 1

for c in clube:
  variaveis[c] = i
  i += 1

def regras_principais(s):

  s.add_clause([-variaveis[bigode],variaveis[casado]])
  s.add_clause([variaveis[ribeirao],variaveis[amarela]])
  s.add_clause([-variaveis[casado],-variaveis[domingo]])
  s.add_clause([variaveis[ribeirao],-variaveis[domingo]])
  s.add_clause([variaveis[domingo],-variaveis[ribeirao]])
  s.add_clause([variaveis[bigode],-variaveis[amarela]])
  s.add_clause([-variaveis[ribeirao],variaveis[bigode]])

In [ ]:
s = Minisat22()
regras_principais(s)

if s.solve():
  m = s.get_model()
  print(m)
  print("\nSAT => O conjunto é consistente! \n")

  print("Um sócio que: ")
  for c in clube:
    if (m[variaveis[c] - 1] > 0):
      
      print(" -> " + c)
    else:
      print(" -> não " + c)
  print("satisfaz o conjunto formado por todas as regras!")
else:
  print("UNSAT => O conjunto é inconsistente.")

s.delete()

[1, 2, -3, 4, -5]

SAT => O conjunto é consistente! 

Um sócio que: 
 -> tem bigode
 -> é casado
 -> não é de Ribeirão
 -> usa camisola amarela
 -> não assiste jogos aos domingos
satisfaz o conjunto formado por todas as regras!


Executando o código acima, verificamos que o conjunto de regras é consistente, visto que existe pelo menos uma atribuição que satisfaz a conjunção de todas as regras, o que implica que satisfaz cada uma das regras individualmente.

###3. Use agora o SAT solver para o ajudar a responder às seguintes questões:

Para sabermos a resposta a cada uma das seguintes questões é necessário adicionar novas cláusulas ao programa capazes de exprimir as afirmações.   
De salientar que entre cada questão é necessário apagar as cláusulas adicionadas das questões anteriores. 

## **(a) A afirmação “Quem usa bigode não pode ir ao jogo ao Domingo.” é correcta?**



A afirmação acima pode ser expressa pela seguinte cláusula: **B → ¬D**.   
Em CNF temos:  **¬B ∨ ¬D**.   
Negação: **B ∧ D**.  
  
Se o conjunto de regras acrescido da negação da cláusula que representa a afirmação for insatisfazível, significa que não existe nenhuma solução em que quem usa bigode possa ir ao jogo ao domingo, ou seja, que, para todas as soluções, a afirmação verifica-se.  


In [ ]:
s = Minisat22()
regras_principais(s)

s.add_clause([variaveis[bigode]])
s.add_clause([variaveis[domingo]])

if s.solve():
  m = s.get_model()
  print(m)
  print("\nSAT => A afirmação é incorreta. Existe pelo menos uma solução em que quem usa bigode pode ir ao jogo no domingo.")

  print("Uma solução para este conjunto de regras é um sócio que: ")
  for c in clube:
    if (m[variaveis[c] - 1] > 0):    
      print(" -> " + c)
    else:
      print(" -> não " + c)

else:
  print("UNSAT => A afirmação é correta, quem usa bigode não pode ir ao jogo ao domingo.")

s.delete()

UNSAT => A afirmação é correta, quem usa bigode não pode ir ao jogo ao domingo.


##**(b) Pode um membro de camisola amarela ser casado?**

A pergunta traduz-se na regra: **A ∧ C**.  
Neste caso só é necessário saber se o conjunto de regras acrescido desta cláusula tem pelo menos uma solução, pois isso significará que existe a possibilidade de um membro usar camisola amarela e ser casado.

In [ ]:
s = Minisat22()
regras_principais(s)

s.add_clause([variaveis[amarela]])
s.add_clause([variaveis[casado]])

if s.solve():
  m = s.get_model()
  print(m)
  print("\nSAT => A afirmação é correta, um membro de camisola amarela pode ser casado. \n")

  print("Uma solução para este conjunto de regras é um sócio que: ")
  for c in clube:
    if (m[variaveis[c] - 1] > 0):    
      print(" -> " + c)
    else:
      print(" -> não " + c)

else:
  print("UNSAT => A afirmação é incorreta. Não existe nenhuma solução em que um membro de camisola amarela é casado.")

s.delete()


[1, 2, -3, 4, -5]

SAT => A afirmação é correta, um membro de camisola amarela pode ser casado. 

Uma solução para este conjunto de regras é um sócio que: 
 -> tem bigode
 -> é casado
 -> não é de Ribeirão
 -> usa camisola amarela
 -> não assiste jogos aos domingos


##**(c) A afirmação “Afinal o clube não pode ter sócios Ribeironenses.” é correcta?**

A afirmação traduz-se na regra **¬R**.  
Negação: **R**.  
  
Para responder à pergunta, adicionamos ao conjunto de regras a cláusula **R**. Se o problema for satisfazível, significa que existe pelo menos uma solução em que os sócios são ribeironenses, o que prova que a afirmação é incorreta. Caso o problema seja insatisfazível, significa que não existe nenhuma solução em que os sócios são ribeironeneses, ou seja, para todas as soluções do problema os sócios não são ribeironenses.

In [ ]:
s = Minisat22()
regras_principais(s)

s.add_clause([variaveis[ribeirao]])

if s.solve():
  m = s.get_model()
  print(m)
  print("\nSAT => A afirmação é incorreta, um sócio pode ser ribeironense. \n")

  print("Uma solução para este conjunto de regras é um sócio que: ")
  for c in clube:
    if (m[variaveis[c] - 1] > 0):    
      print(" -> " + c)
    else:
      print(" -> não " + c)

else:
  print("UNSAT => A afirmação é correta, o clube não tem sócios ribeironenses.")

s.delete()

UNSAT => A afirmação é correta, o clube não tem sócios ribeironenses.


##**(d) Os sócios casados têm todos bigode?**

A pergunta pode traduzir-se na regra: **C → B**.  
Em CNF: **¬C ∨ B**.  
Negação: **C ∧ ¬B**.  
  
Para responder à pergunta adicionamos ao conjunto de regras a negação da regra que exprime a pergunta. Caso esse conjunto de regras seja satisfazível, significa que existe pelo menos uma solução em que os sócios são casados e não têm bigode. Caso o resultado seja UNSAT, significa que não existe nenhuma solução em que não se verifique que os sócios casados têm bigode.

In [ ]:
s = Minisat22()
regras_principais(s)

s.add_clause([variaveis[casado]])
s.add_clause([-variaveis[bigode]])

if s.solve():
  m = s.get_model()
  print(m)
  print("\nSAT => Podem existir sócios que não têm bigode.\n")

  print("Uma solução para este conjunto de regras é um sócio que: ")
  for c in clube:
    if (m[variaveis[c] - 1] > 0):    
      print(" -> " + c)
    else:
      print(" -> não " + c)

else:
  print("UNSAT => Sim, todos os sócios casados têm bigode.")

s.delete()

UNSAT => Sim, todos os sócios casados têm bigode.


##**(e) A afirmação “Ao domingo nunca há sócios a assistir aos jogos.” é correcta?**

A afirmação pode traduzir-se na regra: **¬D**.  
Negação: **D**.  
  
Para responder à pergunta adicionamos ao conjunto de regras a negação da regra que exprime a afirmação. Caso esse conjunto regras seja satisfazível, significa que existe pelo menos uma solução em que ao domingo há sócios a assistir aos jogos. Caso a resolução do problema dê UNSAT, significa que não existe nenhuma solução em que a negação da afirmação se verifique, ou seja, que para todas as soluções ao domingo nunca há sócios a assistir aos jogos.

In [ ]:
s = Minisat22()
regras_principais(s)

s.add_clause([variaveis[domingo]])

if s.solve():
  m = s.get_model()
  print(m)
  print("\nSAT => A afirmação é incorreta. Podem existir sócios a assistir jogos ao domingo. \n")

  print("Uma solução para este conjunto de regras é um sócio que: ")
  for c in clube:
    if (m[variaveis[c] - 1] > 0):    
      print(" -> " + c)
    else:
      print(" -> não " + c)
      
else:
  print("UNSAT => A afirmação é correta, nunca há sócios a assistir aos jogos no domingo.")

s.delete()

UNSAT => A afirmação é correta, nunca há sócios a assistir aos jogos no domingo.


###A resposta à maioria das questões anteriores obteve-se traduzindo as perguntas/afirmações em cláusulas, negando-as e adicionando-as ao conjunto de regras. Ao negarmos a cláusula e resolvendo o problema, obtemos SAT caso exista uma solução em que a afirmação é falsa e obtemos UNSAT caso não exista nenhuma solução em que a negação da afirmação seja verdade, ou seja, a  afirmação é verdadeira para qualquer solução do problema, que é o mesmo que dizer que **a afirmação é válida**.

Podemos verificar as respostas às perguntas calculando todas as soluções possíveis do problema:

In [ ]:
i = 1
new_clause = [0,0,0,0,0]
s = Minisat22()
regras_principais(s)

while s.solve():
  j = 0
  m = s.get_model()
  print("\nSolução " + str(i))
  print(m)

  for c in clube:
    if (m[variaveis[c] - 1] > 0):    
      print(" -> " + c)
      new_clause[j] = -variaveis[c]
      j += 1

    else:
      print(" -> não " + c)
      new_clause[j] = variaveis[c]
      j += 1

  s.add_clause(new_clause)
  i += 1


print("\nSem mais soluções\n")
s.delete()


Solução 1
[1, 2, -3, 4, -5]
 -> tem bigode
 -> é casado
 -> não é de Ribeirão
 -> usa camisola amarela
 -> não assiste jogos aos domingos

Sem mais soluções

